In [1]:
import rasterio
import numpy as np

In [2]:
VIIRS_INPUT = r"C:\Users\Ankit\Datasets_Forest_fire\updated_viirs_binary_fire_2015_2016.nc"
OUT_FILE = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif"

In [3]:
n_bands = 17544   # hourly for 2015–2016
with rasterio.open(VIIRS_INPUT) as src:
    height, width = src.height, src.width
    src_data = src.read(1)  
    profile = src.profile
    profile.update(
        count=n_bands,
        dtype="uint8",
        driver="GTiff"
    )

    with rasterio.open(OUT_FILE, "w", **profile) as dst:
        for i in range(n_bands):
           
            dst.write(src_data.astype("uint8"), i + 1)

In [4]:
print(f"✅ Created VIIRS time stack with {n_bands} bands → {OUT_FILE}")

✅ Created VIIRS time stack with 17544 bands → C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif
